In [ ]:
# %load init.py
import os
import pickle
import sys
# Enable module import from the parent directory from notebooks
sys.path.append(os.path.abspath('..'))
import time

import matplotlib as mpl
# Select plotting backend
mpl.use('nbAgg')

import matplotlib.pyplot as plt
# Customize plotting
plt.style.use('seaborn-paper')
plt.rcParams['axes.labelsize'] = 11.0
plt.rcParams['axes.titlesize'] = 12.0
plt.rcParams['errorbar.capsize'] = 3.0
plt.rcParams['figure.dpi'] = 72.0
plt.rcParams['figure.titlesize'] = 12.0
plt.rcParams['legend.fontsize'] = 10.
plt.rcParams['lines.linewidth'] = 1.
plt.rcParams['xtick.labelsize'] = 11.0
plt.rcParams['ytick.labelsize'] = 11.0

import numpy as np
import sympy as sp
sp.init_printing(euler=True, use_latex=True)

from IPython import display
from scipy import io, optimize
from sklearn import metrics

import core
import dynamicals
import kernels
import numericals
import utils

In [ ]:
dynamical = dynamicals.LotkaVolterra()

spl_t_0, spl_t_T, spl_freq = 0, 2, 100
obs_t_0, obs_t_T, obs_freq = 0, 2, 10
est_t_0, est_t_T, est_freq = 0, 2, 10
spl_tps, obs_tps, obs_t_indices, est_tps, est_t_indices = utils.create_time(
    spl_t_0, spl_t_T, spl_freq, obs_t_0, obs_t_T, obs_freq, est_t_0, est_t_T, est_freq)
X_0 = np.array([5., 3.]) 
theta = np.array([2., 1., 1., 4.]) 
rho_2 = None
phi = [
    # (Kernal name, Kernal parameters)
    ('rbf', np.sqrt([2.5, 0.02])),
    ('rbf', np.sqrt([2.5, 0.02]))
]
sigma_2 = np.array([0.1, 0.1]) 
delta = np.full(dynamical.num_x, True)
gamma = np.array([5e-3, 5e-3]) 

opt_method = 'Newton-CG'
opt_tol = 1e-6
max_init_iter = None
max_iter = 2000

plotting_enabled = True
plotting_freq = 50
plotting_config = None

In [ ]:
dynamical = dynamicals.ProteinSignallingTransduction() 

spl_t_0, spl_t_T, spl_freq = 0, 100, 20
obs_t_0, obs_t_T, obs_freq = 0, 100, 20
est_t_0, est_t_T, est_freq = 0, 100, 20
spl_tps, obs_tps, obs_t_indices, est_tps, est_t_indices = utils.create_time(
    spl_t_0, spl_t_T, spl_freq, obs_t_0, obs_t_T, obs_freq, est_t_0, est_t_T, est_freq)
t_indices = np.array([0, 1, 2, 4, 5, 7, 10, 15, 20, 30, 40, 50, 60, 80, 100]) * spl_freq
obs_tps = obs_tps[t_indices]
obs_t_indices = obs_t_indices[t_indices]
est_tps = est_tps[t_indices]
est_t_indices = est_t_indices[t_indices]
X_0 = np.array([1., 0., 1., 0., 0.]) 
theta = np.array([0.07, 0.6, 0.05, 0.3, 0.017, 3.]) 
rho_2 = None
sigma_2 = np.full(dynamical.num_x, 1e-2) 
delta = np.full(dynamical.num_x, True) 
# gamma = np.full(dynamical.num_x, 5e-3) 
gamma = np.array([1e-4, 1e-4, 2e-6, 1e-5, 5e-2])
phi = [
    # (Kernal name, Kernal parameters)
    ('sigmoid', np.array([1., .4, 15.])),
    ('sigmoid', np.array([.18, .6, 25.])),
    ('sigmoid', np.array([.84, 3., 3.1])),
    ('sigmoid', np.array([.62, 2., 2.1])),
    ('sigmoid', np.array([.84, 3., 2.9])),
]

opt_method = 'Newton-CG'
opt_tol = 1e-6
max_init_iter = None
max_iter = 2000

plotting_enabled = True
plotting_freq = 50
plotting_config = {
    'x': {
        'xlim': (0, 100),
        'ylim': (0., 1.2)
    }
}

In [ ]:
spl_X = dynamical.generate_sample_path(theta, rho_2, X_0, spl_tps)

In [ ]:
obs_Y = utils.collect_observations(spl_X, obs_t_indices, sigma_2, [0, None])

mu, Sigma, inv_Sigma, m, inv_Lambda, Lambda = core.init_with_gaussian_processes(dynamical, 
                                                                                obs_Y, obs_tps, 
                                                                                est_tps,
                                                                                phi, sigma_2, delta, gamma,
                                                                                plotting_enabled)

utils.plot_states(dynamical, spl_X, spl_tps, obs_Y, obs_tps, delta, mu, est_tps, plotting_config)

In [ ]:
num_x = dynamical.num_x
num_theta = dynamical.num_theta
num_est_t = est_tps.size

X_sym = sp.Matrix(
    list(sp.symbols('{}[(:{})][(:{})]'.format(dynamical.x_label, num_x, num_est_t), real=True))
).reshape(num_x, num_est_t)

theta_sym = sp.Matrix(
    dynamical.theta
)

F_sym = sp.Matrix.hstack(*[
    dynamical.F.xreplace(dict(zip(dynamical.x, X_sym.col(t))))
    for t in range(num_est_t)
])

mx_sym = sp.Matrix.hstack(*[
    sp.Matrix(m[i]) * X_sym.row(i).T
    for i in range(num_x)
]).T

F_sym_mx_sym = F_sym - mx_sym
common_objectives_sym = []
for i in range(num_x):
    print('Constructing common objective for state {}.'.format(i + 1))
    common_objectives_sym.append(
        sp.expand(0.5 * F_sym_mx_sym.row(i) * sp.Matrix(Lambda[i]) * F_sym_mx_sym.row(i).T)
    )
    
X_sym_mu = X_sym - sp.Matrix(mu)
x_objectives_sym = []
for i in range(num_x):
    print('Constructing objective for state {}.'.format(i + 1))
    x_objectives_sym.append(
        sp.expand(0.5 * X_sym_mu.row(i) * sp.Matrix(inv_Sigma[i]) * X_sym_mu.row(i).T)
    )
    
x_gradients_sym = []
for i in range(num_x):
    print('Constructing gradient for state {}.'.format(i + 1))    
    syms = [
        sym.jacobian(X_sym.row(i)) 
        for sym in common_objectives_sym
    ]
    syms.append(x_objectives_sym[i].jacobian(X_sym.row(i))) 
    x_gradients_sym.append(syms)
    
x_hessians_sym = []
for i in range(num_x):
    print('Constructing hessian for state {}.'.format(i + 1))
    syms = [
        sym.jacobian(X_sym.row(i))
        for sym in x_gradients_sym[i]
    ]
    x_hessians_sym.append(syms) 
    
theta_gradients_sym = []
for i in range(num_x):
    print('Constructing gradient for theta over state {}'.format(i + 1))
    theta_gradients_sym.append(common_objectives_sym[i].jacobian(theta_sym))

theta_hessians_sym = []
for i in range(num_x):
    print('Constructing hessian for theta over state {}'.format(i + 1))
    theta_hessians_sym.append(theta_gradients_sym[i].jacobian(theta_sym))

In [ ]:
try:
    print('Constructing common objective functions.')
    common_objectives_func = [
        sp.lambdify((dynamical.dummy_x, dynamical.dummy_theta), sym, dummify=False, modules='numpy')
        for sym in common_objectives_sym
    ]

    print('Constructing objetive functions for states.')
    x_objectives_sym_func = [
        sp.lambdify((dynamical.dummy_x, dynamical.dummy_theta), sym, dummify=False, modules='numpy')
        for sym in x_objectives_sym
    ]

    print('Constructing gradient functions for states.')
    x_gradients_sym_func = [
        [
            sp.lambdify((dynamical.dummy_x, dynamical.dummy_theta), sym, dummify=False, modules='numpy')
            for sym in x_gradients_sym[i]
        ]
        for i in range(num_x)
    ]

    print('Constructing hessian functions for states.')
    x_hessians_sym_func = [
        [
            sp.lambdify((dynamical.dummy_x, dynamical.dummy_theta), sym, dummify=False, modules='numpy')
            for sym in x_hessians_sym[i]
        ]
        for i in range(num_x)
    ]
    
    print('Constructing gradient functions for theta.')
    theta_gradients_sym_func = [
        sp.lambdify((dynamical.dummy_x, dynamical.dummy_theta), sym, dummify=False, modules='numpy')
        for sym in theta_gradients_sym
    ]
    
    print('Constructing hessian functions for theta.')
    theta_hessians_sym_func = [
        sp.lambdify((dynamical.dummy_x, dynamical.dummy_theta), sym, dummify=False, modules='numpy')
        for sym in theta_hessians_sym
    ]
    
    def x_objective_and_gradient_(x):
        eta_X[i] = x
        objective = x_objectives_sym_func[i](eta_X, eta_theta).ravel()[0]
        for func in common_objectives_func:
            objective += func(eta_X, eta_theta).ravel()[0]

        gradient = np.zeros(num_est_t)
        for func in x_gradients_sym_func[i]:
            gradient += func(eta_X, eta_theta).ravel()
        return objective, gradient

    def x_hessian_(x):
        hessian = np.zeros((num_est_t, num_est_t))
        for func in x_hessians_sym_func[i]:
            hessian += func(eta_X, eta_theta)
        return hessian

    def theta_objective_and_gradient_(theta):
        eta_theta[:] = theta
        objective = 0
        for func in common_objectives_func:
            objective += func(eta_X, theta).ravel()[0]

        gradient = np.zeros(num_theta)
        for func in theta_gradients_sym_func:
            gradient += func(eta_X, theta).ravel()
        return objective, gradient

    def theta_hessian_(theta):
        hessian = np.zeros((num_theta, num_theta))
        for func in theta_hessians_sym_func:
            hessian += func(eta_X, theta)
        return hessian    
except RecursionError as e:
    raise RuntimeError('Unable to construct functions')    

In [ ]:
eta_X = mu.copy()
Xi_X = Sigma.copy()
eta_theta = np.zeros(num_theta)
Xi_theta = np.zeros((num_theta, num_theta))

figure, plotting_config = utils.create_estimation_figure(dynamical, delta)
utils.add_sample_path(figure, plotting_config, spl_X, spl_tps)
utils.add_observations(figure, plotting_config, obs_Y, obs_tps, delta)
utils.add_gaussian_regression_result(figure, plotting_config, eta_X, est_tps)

eta_theta_costs = []
eta_X_costs = []

for it in range(1, 1001):
    result = optimize.minimize(fun=theta_objective_and_gradient_, x0=eta_theta, method='Newton-CG', 
                               jac=True, hess=theta_hessian_, options={'disp': False})
#     eta_theta = np.abs(result.x)
    eta_theta_cost = result.fun
    eta_theta_costs.append(eta_theta_cost)
    
    eta_X_cost = 0
    for i in range(num_x):
        result = optimize.minimize(fun=x_objective_and_gradient_, x0=eta_X[i], method='Newton-CG', 
                                   jac=True, hess=x_hessian_, options={'disp': False})
        eta_X_cost += result.fun
    eta_X_costs.append(eta_X_cost)
    
    if it % 50 != 0:
        print('.', end='')
    else:
        print('Iteration: {}, theta cost: {:.4f}, X cost: {:.4f}'.format(it, eta_theta_cost, eta_X_cost))
        utils.add_estimation_step(figure, plotting_config, dynamical, eta_X, est_tps, theta, eta_theta)
        
for i in range(num_x):
    x_objective_and_gradient_(eta_X[i])
    Xi_X[i] = numericals.cholesky_inv(x_hessian_(eta_X[i]))
    
    theta_objective_and_gradient_(eta_theta)
    Xi_theta = numericals.cholesky_inv(theta_hessian_(eta_theta))

utils.add_estimation_result(figure, plotting_config, dynamical, eta_X, est_tps, theta, eta_theta)
utils.plot_estimation_result(plotting_config, dynamical, spl_X, spl_tps, obs_Y, obs_tps, delta, eta_X, Xi_X,
                             est_tps, theta, eta_theta, Xi_theta)        

In [ ]:
# Helper to plot costs
figure = plt.figure(figsize=plt.figaspect(0.4))
ax = figure.add_subplot(1, 2, 1)
ax.plot(eta_X_costs)
ax.set_title('Cost for X')

ax = figure.add_subplot(1, 2, 2)
ax.plot(eta_X_costs)
ax.set_title('Cost for theta')

figure.tight_layout()
plt.show()

In [ ]:
# Helper to plot theta for protein signaling transduction
figure = plt.figure()
ax = plt.gca()
bar_width = 0.3
bar_indices = np.arange(theta.size - 1)
ax.bar(bar_indices, theta[:-1], bar_width, color='C0', label='Truth')
ax.bar(bar_indices + bar_width, eta_theta[:-1], bar_width, color='C2', label='Estimation',
       yerr=2 * np.sqrt(np.diagonal(Xi_theta))[:-1],
       error_kw=dict(ecolor='0.2', capsize=3., capthick=1.))
ax.set_ylabel('Value')
ax.set_title('Theta')
ax.set_xticks(bar_indices + bar_width / 2)
ax.set_xticklabels([r'${}$'.format(label) for label in dynamical.theta_labels])
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, loc=0)
figure.tight_layout()
plt.show()